<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/pull_one_hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()

#filter = {
       # "MlsStatus": "SLD",
       # "ModificationTimestamp": {"$gte": dt_string}
        #}
#result = rets_client.search(resource='Property', resource_class='Property', limit = 10,search_filter=filter)





In [276]:
# low price
tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 1)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")


lowerPrice_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(ModificationTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))


for item in lowerPrice_result:
  if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['PreviousListPrice']>item['ListPrice']):
    df = df.append(item, ignore_index=True)
df = pd.DataFrame(df)

for item in df['ListPrice']:
  item = float(item)
for item in df['PreviousListPrice']:
  item = float(item) # has empty string

percent = (item['PreviousListPrice'] - item['ListPrice'] ) /item['ListPrice']

#df.to_csv('low_price.csv')


ValueError: ignored

In [275]:
#Relist - Back on Market

tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 1)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")

relist_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(ModificationTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
df = pd.DataFrame()
for item in relist_result:
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['PreviousStatus'] == "AOC" or item['PreviousStatus'] == "Expired" or item['PreviousStatus'] == "Active Option Contract" or item['PreviousStatus'] == "Withdrawn"):
      df = df.append(item, ignore_index=True)
df = pd.DataFrame(df)

#add new column

  
df.to_csv('BOM.csv')

ValueError: ignored

In [274]:
df['ListPrice']

0      498000.00
1      880000.00
2     1050000.00
3       48000.00
4      850000.00
5      279000.00
6      485000.00
7      675000.00
8      389000.00
9      299000.00
10     975000.00
11    2995000.00
12     269000.00
13     429900.00
Name: ListPrice, dtype: object

TypeError: ignored